# Tinglysningsdata
Script som beriger matrikeltabel med oplysninger fra tingbogen ud fra matrikeloplysninger som slås op i tingbogens [system til system adgang](http://www.tinglysningsretten.dk/etl/Pages/default.aspx) som kan tilgås via [API'et](http://www.tinglysningsretten.dk/etl/hoveddokumenter/Documents/HTTP_API_beskrivelse%20v1.1.docx). Scriptet er opbygget i tre faser:
1. [Input af matrikeldata fra database](#Input)
2. [Berigelse af matrikeldata med data fra tingbogen](#Berigelse)
3. [Output berigede matrikeldata til database](#Output)

In [2]:
import pandas as pd
import numpy as np
#import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

#### Databaseforbindelsen opsættes

In [ ]:
user = 'xxx'
pw = 'xxx'
port = 5432
host = 'postgres'
db = 'ballerup'
schema = 'proj_tinglysning'
table = 'ballerup'

In [ ]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

# Input
Laver DF med matrikeloplysninger for Ballerup Kommune. Her er listet to muligheder: 1. som henter data fra csv og 2. som henter fra PostgreSQL

### 1. CSV

In [3]:
# Laver DF med matrikeloplysninger for Ballerup Kommune 
# Fjern .head(5) når det hele skal ind 
matrikel = pd.read_csv('data/matrikel.csv').head(10)

In [4]:
matrikel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
elavskode    10 non-null int64
matrnr       10 non-null object
esr_ejdnr    10 non-null int64
dtypes: int64(2), object(1)
memory usage: 320.0+ bytes


In [ ]:
### 2. PostgreSQL

In [ ]:
# Laver DF med matrikeloplysninger for Ballerup Kommune 
# Fjern .head(5) når det hele skal ind 
matrikel = pd.read_sql_query('select elavskode, matrnr, esr_ejdnr from "_00_grundkort"."_00_02_jordstykke"',con=con).head()

In [ ]:
# Sætter datatype for ejelav og ejdnr til heltal
matrikel.elavskode = pd.to_numeric(matrikel.elavskode, errors='coerce').astype(np.int64)
matrikel.esr_ejdnr = pd.to_numeric(matrikel.esr_ejdnr, errors='coerce').astype(np.int64)

In [ ]:
matrikel

# Berigelse
I denne sektion beriges matriklerne med data fra Tinglysning.

In [37]:
#import requests
import time # Bruges til ikke at overbelaste API'et
import OpenSSL.crypto
import json
import xml.etree.ElementTree as etree
import tempfile
import requests
import os

# sti til PEM certifikat
cert = 'data/certifikat.pem'
# ejendomsliste
ejdlist =[]

Tænker at det er i nedenstående funktion vi har brug for hjælp til at få hul igennem til API'et. Hvis der er flere tinglysninger tilknyttet en matrikel tænker jeg at ```get_tinlysning``` kan returnere en liste med json-objekter fra kaldene.
<br><br>
[Beskrivelse af hvordan parameteren row fungerer i pandas](https://stackoverflow.com/a/30389492)


In [38]:
def pfx_to_pem(pfx_path, pfx_password):
    ''' Udpakning af p12 certifikat til PEM  '''
    with tempfile.NamedTemporaryFile(suffix='.pem', delete=False) as t_pem:
        f_pem = open(t_pem.name, 'wb')
        pfx = open(pfx_path, 'rb').read()
        p12 = OpenSSL.crypto.load_pkcs12(pfx, pfx_password)
        f_pem.write(OpenSSL.crypto.dump_privatekey(OpenSSL.crypto.FILETYPE_PEM, p12.get_privatekey()))
        f_pem.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, p12.get_certificate()))
        ca = p12.get_ca_certificates()
        if ca is not None:
            for cert in ca:
                f_pem.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))
        f_pem.close()
        yield t_pem.name


In [39]:
# Henter akt pdf fil
def get_akt(akt):
    r = requests.get(
        'https://www.tinglysning.dk/tinglysning/ssl/indskannetakt/' + akt,
        cert=cert)
    if r.status_code != 200:
        print('http error')
    try:
        aktjson = json.loads(r.text)
    except ValueError:
        print('Decode error')
        fejl = 1
    uuid = aktjson['uuid']
    aktfilnavn =aktjson['filnavn']
    if os.path.isfile('data/akt/'+aktfilnavn) == False:
        r = requests.get(
            'https://www.tinglysning.dk/tinglysning/pdfStreamer?entity=akt&uuid=' + uuid + '&filename=' + aktfilnavn,
            stream=True)
        if r.status_code == 200:
            with open('data/akt/'+aktfilnavn, 'wb') as f:
                for chunk in r.iter_content(2097152): # 2mb chunk size
                    f.write(chunk)

In [40]:
# henter deklaration med tilhørende bilag, for servitutter lyst på den digitale tinglysning.
# Deklarationer hentes som xml fil og navngives dato-løbenummer.xml
# bilag navngives med bilagsrefencen (guid)
def get_deklaration(datolbnr):

    r = requests.get(
        'https://www.tinglysning.dk/tinglysning/ssl/dokaktuel/alias/' + datolbnr,
        cert=cert)
    deklarationxml = etree.fromstring(r.content)
    file = open("data/bilag/" + datolbnr +".xml","wb")
    file.write(etree.tostring(deklarationxml,encoding='utf8',method='xml'))

    #tingbogsparser(tingxml)
    bilaglist = deklarationxml.findall('.//{http://rep.oio.dk/tinglysning.dk/schema/model/1/}BilagReference')
    for bilag in bilaglist:
        try:
            if len(bilag) >0:
                fileext = ''
                if bilag[1].text == '80':
                    #pdf fil
                    fileext = '.pdf'
                if bilag[1].text == '38':
                    # gml filer
                    fileext = '.gml'

                if os.path.isfile('data/bilag/' + bilag[0].text + fileext) == False:
                    r = requests.get(
                        'https://www.tinglysning.dk/tinglysning/ssl/bilag/' + bilag[0].text,
                        cert=cert)
                    #    https://www.tinglysning.dk/tinglysning/rest/bilag/b06a3da3-a75f-4c1d-aa75-e2c881d077b3
                    if r.status_code == 200:
                        with open('data/bilag/' + bilag[0].text + fileext, 'wb') as f:
                            for chunk in r.iter_content(2097152):  # 2mb chunk size
                                f.write(chunk)
        except IndexError:
            print('fejl bilagsreference')


In [41]:
def tingbogsparser(tingxml):
    #Fra tinglysningen kommer der xml fil retur med servitutter mm.

    # Start med at løbe stamopslysninger igennem
    # bfenr = bestemt fast ejendom
    # komkode = kommune kode
    # bbrnr = bbr nummmer
    # ejdtype = ejendommens notering
    for elem in tingxml.findall('.//{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}EjendomStamoplysninger'):
        for child in elem.getchildren():
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}EjendomIdentifikator':
                bfenr = child[0].text
            # den scannede akt hentes
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}EjendomIndskannetAktSamling':
                ejdakt = child[0].text
                get_akt(ejdakt)

            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/snapshot/1/}EjendomVurderingSamling':
                bbrnr = child[0][0][1].text
                komkode = child[0][0][0].text

            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}HovedNotering':
                ejdtype = child.text

    # servitutter udlæses
    for elem in tingxml.findall('.//{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}ServitutSummarisk'):
        for child in elem.getchildren():
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}DokumentRevisionIdentifikator':
                dokid = child[0].text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}TinglysningsDato':
                tingdato = child.text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}SenestPaategnetDato':
                paategndato = child.text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}Servitutrettighed':
                rettighedid = child[0].text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}OgsaaLystPaaSamling':
                ogsaalystpaa = child[0].text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}DokumentAlias':
                # deklarationer fra før digital tinglysning
                if child[
                    0].tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}AktHistoriskIdentifikator':
                    historiskid = child[0].text
                # deklaration fra digital tinglysning
                # der hentes xml deklaration + evt. gml filer
                if child[
                    0].tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}DokumentAliasIdentifikator':
                    dokalias = child[0].text
                    get_deklaration(dokalias)

            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/model/1/}ServitutType':
                servituttype = child.text
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}TillaegstekstSamling':
                servituttekst = ''.join(child.itertext())
            # den scannede akt hentes
            if child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}DokumentInformationOverfoert':
                dokfilnavn = child[2].text
                get_akt(dokfilnavn)


In [42]:
def get_tinglysning(row):
    """
    Henter data fra tinglysning for hver række i dataframe på baggrund af
    matrikeloplysninger. Bruges i sammenhæng med apply metoden på DF
    """
    ejerlav = str(round(row['elavskode']))
    matnr = row['matrnr']
    #ejedomsnr = row['esr_ejdnr'] #Bruges ikke endnu, men kan bruges som alternativ til matrikelnummer/ejerlav
    # Det anbefales at bruge matrikelnummer/ejerlav.
    url = 'https://www.tinglysning.dk/tinglysning/ssl/ejendom/landsejerlavmatrikel?landsejerlavid={}&matrikelnr={}'.format(ejerlav, matnr)
    r = requests.get(url, cert=cert)
    if r.status_code!= 200:
        print('http error')

    fejl = 0
    try:
        data = json.loads(r.text)
    except ValueError:
        print('Decode error')
        fejl = 1
    #I JSON der returnerers fra tinglysningen er der et uuid for hver ejendom på matriklen. 
    # Der laves tingbogsopslag på hvert uuid, det er XML der returneres
    if data['items'] != None:
        for c in data['items']:
            try:
                idx = ejdlist.index(c['uuid'])
                opslag = False
            except ValueError:
                opslag = True
            if opslag == True:
                r = requests.get(
                'https://www.tinglysning.dk/tinglysning/ssl/ejdsummarisk/'+c['uuid'],
                cert=cert)
                ejdlist.append(c['uuid'])
                tingxml = etree.fromstring(r.content)

                file = open("data/tingbog/" + c['uuid'] +".xml","wb")
                file.write(etree.tostring(tingxml,encoding='utf8',method='xml'))

                tingbogsparser(tingxml)
                # Laver et kald hvert 5 sekund
                time.sleep(5)
    
    return url


In [43]:
matrikel['tinglysning'] = matrikel.apply(get_tinglysning, axis=1)

12
12
6
4


In [44]:
print(matrikel['tinglysning'][1])

https://www.tinglysning.dk/tinglysning/ssl/ejendom/landsejerlavmatrikel?landsejerlavid=20151&matrikelnr=182g


In [45]:
matrikel

,elavskode,matrnr,esr_ejdnr,tinglysning
0,20151,182k,1510135021,https://www.tinglysning.dk/tinglysning/ssl/eje...
1,20151,182g,1510149184,https://www.tinglysning.dk/tinglysning/ssl/eje...
2,20151,7000bd,0,https://www.tinglysning.dk/tinglysning/ssl/eje...
3,20151,11bø,1510034160,https://www.tinglysning.dk/tinglysning/ssl/eje...
4,20151,11cz,1510034152,https://www.tinglysning.dk/tinglysning/ssl/eje...
5,20151,11cx,1510034101,https://www.tinglysning.dk/tinglysning/ssl/eje...
6,20151,11cæ,1510034144,https://www.tinglysning.dk/tinglysning/ssl/eje...
7,20151,11cp,1510017479,https://www.tinglysning.dk/tinglysning/ssl/eje...
8,20151,11cy,1510034136,https://www.tinglysning.dk/tinglysning/ssl/eje...
9,20151,7000bb,0,https://www.tinglysning.dk/tinglysning/ssl/eje...


# Output
Der er lavet to muligheder for output, CSV og PostgreSQL

### 1. CSV

In [46]:
matrikel.to_csv('data/output.csv')

### 2. PostgreSQL

In [47]:
dtype = {
    'elavskode': sqlalchemy.VARCHAR(), 
    'matrnr': sqlalchemy.VARCHAR(),
    'esr_ejdnr': sqlalchemy.VARCHAR(),
    'tinglysning': sqlalchemy.JSON()
}

In [48]:
matrikel.to_sql(table, con, schema=schema, if_exists='replace', index_label='gid', dtype=dtype)

NameError: name 'table' is not defined